In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False
rcParams["font.family"] = 'DejaVu Sans'
rcParams["font.size"] = 12

# DF experiments

In [ ]:
exp_name = "2021_10_07_stepper";  # new hardware, new sweep
#exp_name = "2021_07_08_stepper_fast";  # new hardware, new sweep
#exp_name = "2021_07_08_stepper_slow";  # new hardware, new sweep
#exp_name = "2021_07_27_epuck_wall";  # new hardware, new sweep
# new buzzer

motors = "all45000"
bin_selection = 5

#motors = "sweep_and_move" 
#bin_selection = 0

mic_type = "audio_deck" 
source = "sweep_new"
fname = f'plots/experiments/{exp_name}_{motors}_{bin_selection}'

## spectrogram analysis

In [ ]:
# get first distance measurement from the chosen datatype. 
all_data = pd.read_pickle(f'../experiments/{exp_name}/all_data.pkl')
row_idx = 1
data = all_data.loc[(all_data.mic_type==mic_type) & 
                    (all_data.motors==motors) &
                    (all_data.bin_selection==bin_selection)].iloc[row_idx]
data

In [ ]:
print(data.frequencies_matrix.shape)
print(data.stft.shape)
for i in range(data.frequencies_matrix.shape[0]):
    plt.figure()
    plt.title(f'sweep{i}')
    plt.scatter(range(data.frequencies_matrix.shape[1]), data.frequencies_matrix[i, :])
    plt.xlabel('time idx')
    plt.ylabel('chosen frequency [Hz]')
    
    plt.figure()
    plt.plot(data.frequencies_matrix[i, :], np.abs(data.stft[i, 0]))

In [ ]:
from frequency_analysis import get_spectrogram_raw
spec, freqs = get_spectrogram_raw(data.frequencies_matrix, data.stft)
print(len(freqs))
fig, ax = plt.subplots()
for i in range(spec.shape[2]):
    ax.scatter(freqs[freqs>0], spec[freqs>0, 0, i], label=f'sweep{i}')
    #ax.plot(freqs[freqs>0], spec[freqs>0, 0, i], label=f'sweep{i}')
ax.set_yscale('log')
ax.legend()
#ax.set_ylim(0.3, 10)
ax.set_ylim(1e-3, 100)
ax.set_xlabel('frequency [Hz]')
ax.set_ylabel('amplitude')

In [ ]:
from plotting_tools import pcolorfast_custom, add_colorbar, save_fig
fig, ax = plt.subplots()
fig.set_size_inches(5, 3)
spec = np.abs(spec[freqs>0, 0, :])
im = pcolorfast_custom(ax, data.seconds, freqs[freqs>0], np.log10(spec))#, vmin=1e-5)
add_colorbar(fig, ax, im)
xticks = np.arange(0, max(data.seconds), step=5) 
ax.set_xticks(xticks)
ax.set_xticklabels(xticks)
#yticks = [3000, 4000, 5000]
#ax.set_ylim(3000, 5000)
#ax.set_yticks(yticks)
#ax.set_yticklabels(yticks)
ax.set_xlabel('time [s]')
ax.set_ylabel('frequency [Hz]')
#save_fig(fig, f'{fname}_spec.png')

## raw df analysis

In [ ]:
from data_collector import DataCollector

data_collector = DataCollector(exp_name, mic_type)
print(data_collector.params)
backup_exists = data_collector.fill_from_backup(
    exp_name, mic_type, motors, bin_selection, appendix="_raw"
)
data_collector.remove_nan_rows(verbose=True)

In [ ]:
from plotting_tools import plot_df_matrix, save_fig

print(data_collector.df)
print(data_collector.df.frequency.unique())

matrix, dists, freqs = data_collector.get_df_matrix()
print(matrix.shape, dists.shape, freqs.shape)
fig, axs = plt.subplots(1, matrix.shape[0])
fig.set_size_inches(5 * matrix.shape[0], 5)
for i in range(matrix.shape[0]):
    plot_df_matrix(dists, freqs[freqs>0], matrix[i, freqs>0, :], ax=axs[i])
    axs[i].set_title(f"mic{i}")
axs[0].set_ylabel('raw matrix\n frequency [Hz]')

In [ ]:
data_collector.df
for distance, df_chosen in data_collector.df.groupby('distance'):
    plt.figure()
    for i_mic, df_mic in df_chosen.groupby('mic'):
        i_mic = int(i_mic)
        df_mic = df_mic.sort_values(by='time')
        label=f'mic{i_mic}, N={len(df_mic)}'
        plt.scatter(df_mic.time, df_mic.frequency.values, color=f'C{int(i_mic)}', 
                    label=label)
    plt.title(f'distance {distance}')
    plt.xlabel('time')
    plt.ylabel('peak frequency')
    plt.legend(loc='upper left')
    break

In [ ]:
import seaborn as sns

def plot_current_df():
    fig, ax = plt.subplots()
    fig.set_size_inches(5, 5)
    sns.scatterplot(
        data=data_collector.df, x="frequency", y="magnitude", hue="mic", linewidth=0,
        cmap='tab10'
    )
    ax.set_yscale('log')
    #min_y = max(data_collector.df.magnitude.min(), 1e-2)
    #max_y = min(data_collector.df.magnitude.max(), 200)
    #ax.set_ylim(min_y, max_y)
    ax.legend(loc='lower center', ncol=1)

In [ ]:
plot_current_df()

In [ ]:
data_collector.remove_bad_measurements(verbose=True)
plot_current_df()

In [ ]:
data_collector.remove_bad_freqs(verbose=True)
plot_current_df()

In [ ]:
data_collector.merge_close_freqs(verbose=True)
plot_current_df()

In [ ]:
#data_collector.remove_outliers(verbose=True)
plot_current_df()

In [ ]:
data_collector.remove_spurious_freqs(verbose=True)
plot_current_df()

In [ ]:
data_collector.remove_nan_rows(verbose=True)

In [ ]:
from data_collector import prune_df_matrix

matrix_cleaned, dists_cleaned, freqs_cleaned = data_collector.get_df_matrix()
matrix_pruned, freqs_pruned, *_ = prune_df_matrix(matrix_cleaned, freqs_cleaned, ratio_missing_allowed=0.5)

def plot_matrices(dists, freqs, matrix):
    fig, axs = plt.subplots(1, matrix.shape[0], sharey=True)
    fig.set_size_inches(3 * matrix.shape[0], 3)
    for i in range(matrix.shape[0]):
        plot_df_matrix(dists, freqs[freqs>0], matrix[i, freqs>0], ax=axs[i])
        axs[i].set_title(f"mic{i}")
        axs[i].set_ylim(3000, 4505)
    return fig, axs

fig, axs = plot_matrices(dists, freqs, matrix)
axs[0].set_ylabel('raw matrix\n frequency [Hz]')
save_fig(fig, f'{fname}_raw_matrix.png')

fig, axs = plot_matrices(dists_cleaned, freqs_cleaned, matrix_cleaned)
axs[0].set_ylabel('cleaned matrix\n frequency [Hz]')
save_fig(fig, f'{fname}_cleaned_matrix.png')

fig, axs = plot_matrices(dists_cleaned, freqs_pruned, matrix_pruned)
axs[0].set_ylabel('pruned matrix\n frequency [Hz]')
save_fig(fig, f'{fname}_pruned_matrix.png')

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('./../crazyflie-audio/python/'))
from bin_selection import generate_sweep, get_frequencies

bins, *_ =  generate_sweep(source)
freqs_buzzer = get_frequencies()[bins]

if source == "sweep_new":
    freqs = [2999, 3113, 3236, 3370, 3514, 3632, 3757, 3891, 3986, 4137, 4245, 4358, 4477, 4603, 4737, 4878]
    print('error between played and frequency bin:', freqs_buzzer - freqs_cleaned)

plt.figure()
[plt.plot([f, f], [1, 2], color='C0') for f in freqs_buzzer]
[plt.plot([f, f], [2, 3], color='C1') for f in freqs_pruned]
[plt.plot([f, f], [3, 4], color='C2') for f in freqs]
plt.plot([], [], color='C0', label='buzzer') 
plt.plot([], [], color='C1', label='pruned')
plt.plot([], [], color='C2', label='original')
plt.legend()
pass